<a href="https://colab.research.google.com/github/rickiepark/llm-from-scratch/blob/main/ch04/01_main-chapter-code/exercise-solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
세바스찬 라시카(Sebastian Raschka)가 쓴 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>의 번역서 예제 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rickiepark/llm-from-scratch">https://github.com/rickiepark/llm-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 4장 연습문제 해답


In [1]:
from importlib.metadata import version

print("파이토치 버전:", version("torch"))

파이토치 버전: 2.6.0+cu124


# 연습문제 4.1: 피드 포워드 신경망과 어텐션 모듈에 있는 파라미터 수


In [3]:
# 깃허브에서 gpt.py 파일을 다운로드합니다.
!wget https://bit.ly/3SZbhlj -O gpt.py

--2025-06-06 06:17:56--  https://bit.ly/3SZbhlj
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch04/01_main-chapter-code/gpt.py [following]
--2025-06-06 06:17:57--  https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch04/01_main-chapter-code/gpt.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9675 (9.4K) [text/plain]
Saving to: ‘gpt.py’

gpt.py              100%[===================>]   9.45K  --.-KB/s    in 0s      

2025-06-06 06:17:57 (80.7 MB/s) - ‘gpt.py’ saved [9675/9675]



In [4]:
from gpt import TransformerBlock

GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

block = TransformerBlock(GPT_CONFIG_124M)
print(block)

TransformerBlock(
  (att): MultiHeadAttention(
    (W_query): Linear(in_features=768, out_features=768, bias=False)
    (W_key): Linear(in_features=768, out_features=768, bias=False)
    (W_value): Linear(in_features=768, out_features=768, bias=False)
    (out_proj): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (ff): FeedForward(
    (layers): Sequential(
      (0): Linear(in_features=768, out_features=3072, bias=True)
      (1): GELU()
      (2): Linear(in_features=3072, out_features=768, bias=True)
    )
  )
  (norm1): LayerNorm()
  (norm2): LayerNorm()
  (drop_shortcut): Dropout(p=0.1, inplace=False)
)


In [5]:
total_params = sum(p.numel() for p in block.ff.parameters())
print(f"피드 포워드 모듈의 총 파라미터 수: {total_params:,}")

피드 포워드 모듈의 총 파라미터 수: 4,722,432


In [6]:
total_params = sum(p.numel() for p in block.att.parameters())
print(f"어텐션 모듈의 총 파라미터 수: {total_params:,}")

어텐션 모듈의 총 파라미터 수: 2,360,064


- 위는 하나의 트랜스포머 블록에 대한 결과입니다.
- 1억 2,400만 파라미터의 GPT 모델의 경우 위 결과에 12를 곱하면 전체 트랜스포머 블록의 파라미터 수를 구할 수 있습니다.

**보너스: 파라미터 계산 방법**

- 파라미터 계산 방법이 궁금한 사람을 위해 직접 계산해 보죠(`emb_dim=768`이라 가정합니다):

피드 포워드 모듈:

- 첫 번째 `Linear` 층: 768 입력 × 4×768 출력 + 4×768 편향 유닛 = 2,362,368
- 두 번째 `Linear` 층: 4×768 입력 × 768 출력 + 768 편향 유닛 = 2,360,064
- 총:  첫 번째 `Linear` 층 + 두 번째 `Linear` 층 = 2,362,368 + 2,360,064 = 4,722,432

어텐션 모듈:

- `W_query`: 768 입력 × 768 출력 = 589,824
- `W_key`: 768 입력 × 768 출력 = 589,824
- `W_value`: 768 입력 × 768 출력 = 589,824
- `out_proj`: 768 입력 × 768 출력 + 768 편향 유닛 = 590,592
- 총: `W_query` + `W_key` + `W_value` + `out_proj` = 3×589,824 + 590,592 = 2,360,064

# 연습문제 4.2: 더 큰 GPT 모델 초기화하기


- **GPT2-small** (본문에서 구현한 1억 2,400만 모델):
    - "emb_dim" = 768
    - "n_layers" = 12
    - "n_heads" = 12

- **GPT2-medium:**
    - "emb_dim" = 1024
    - "n_layers" = 24
    - "n_heads" = 16

- **GPT2-large:**
    - "emb_dim" = 1280
    - "n_layers" = 36
    - "n_heads" = 20

- **GPT2-XL:**
    - "emb_dim" = 1600
    - "n_layers" = 48
    - "n_heads" = 25

In [7]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}


def get_config(base_config, model_name="gpt2-small"):
    GPT_CONFIG = base_config.copy()

    if model_name == "gpt2-small":
        GPT_CONFIG["emb_dim"] = 768
        GPT_CONFIG["n_layers"] = 12
        GPT_CONFIG["n_heads"] = 12

    elif model_name == "gpt2-medium":
        GPT_CONFIG["emb_dim"] = 1024
        GPT_CONFIG["n_layers"] = 24
        GPT_CONFIG["n_heads"] = 16

    elif model_name == "gpt2-large":
        GPT_CONFIG["emb_dim"] = 1280
        GPT_CONFIG["n_layers"] = 36
        GPT_CONFIG["n_heads"] = 20

    elif model_name == "gpt2-xl":
        GPT_CONFIG["emb_dim"] = 1600
        GPT_CONFIG["n_layers"] = 48
        GPT_CONFIG["n_heads"] = 25

    else:
        raise ValueError(f"잘못된 모델 이름입니다 {model_name}")

    return GPT_CONFIG


def calculate_size(model):

    total_params = sum(p.numel() for p in model.parameters())
    print(f"전체 파라미터 수: {total_params:,}")

    total_params_gpt2 =  total_params - sum(p.numel() for p in model.out_head.parameters())
    print(f"가중치 묶기를 고려한 훈련 가능한 파라미터 수: {total_params_gpt2:,}")

    # 총 크기를 바이트 단위로 계산 (float32, 매개변수당 4바이트 가정)
    total_size_bytes = total_params * 4

    # 메가바이트로 변환
    total_size_mb = total_size_bytes / (1024 * 1024)

    print(f"모델의 총 크기: {total_size_mb:.2f} MB")

In [8]:
from gpt import GPTModel


for model_abbrev in ("small", "medium", "large", "xl"):
    model_name = f"gpt2-{model_abbrev}"
    CONFIG = get_config(GPT_CONFIG_124M, model_name=model_name)
    model = GPTModel(CONFIG)
    print(f"\n\n{model_name}:") # 모델 이름 출력
    calculate_size(model)



gpt2-small:
전체 파라미터 수: 163,009,536
가중치 묶기를 고려한 훈련 가능한 파라미터 수: 124,412,160
모델의 총 크기: 621.83 MB


gpt2-medium:
전체 파라미터 수: 406,212,608
가중치 묶기를 고려한 훈련 가능한 파라미터 수: 354,749,440
모델의 총 크기: 1549.58 MB


gpt2-large:
전체 파라미터 수: 838,220,800
가중치 묶기를 고려한 훈련 가능한 파라미터 수: 773,891,840
모델의 총 크기: 3197.56 MB


gpt2-xl:
전체 파라미터 수: 1,637,792,000
가중치 묶기를 고려한 훈련 가능한 파라미터 수: 1,557,380,800
모델의 총 크기: 6247.68 MB


# 연습문제 4.3: 별도의 드롭아웃 비율 사용하기


In [9]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate_emb": 0.1,        # 임베딩 층에 대한 드롭아웃
    "drop_rate_attn": 0.1,       # 멀티 헤드 어텐션에 대한 드롭아웃
    "drop_rate_shortcut": 0.1,   # 숏컷 연결에 대한 드롭아웃
    "qkv_bias": False
}

In [10]:
import torch.nn as nn
from gpt import MultiHeadAttention, LayerNorm, FeedForward


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate_attn"], # 추가: 멀티 헤드 어텐션에 대한 드롭아웃
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate_shortcut"])

    def forward(self, x):
        # 어텐션 블록을 위한 숏컷 연결
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # 크기 [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # 원래 입력을 다시 더합니다

        # 피드포워드 블록을 위한 숏컷 연결
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # 원래 입력을 다시 더합니다

        return x


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate_emb"]) # 추가: 임베딩 층에 대한 드롭아웃

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # 크기 [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [11]:
import torch

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)